## Механизмы инкапсуляции. Статические методы и методы классов. 

Инкапсуляция в ООП - это сокрытие внутреннего устройства класса от внешних пользователей для того, чтобы взаимодействовать с экземплярами класса можно было только через интерфейс (методы). В питоне готовых инструментов для инкапсуляции, в отличие от объектно-ориентированных языков типа Java, нет, но есть средства их реализации. 

Прежде чем разбираться с тем, как это делается, давайте уясним себе, когда может пригодиться инкапсуляция. Наши с вами классы пока довольно простые, но в реальной жизни класс может быть устроен сложным образом, и у него могут быть какие-то внутренние переменные и методы, которые вызываются в нем самом для обеспечения его работы и не должны вызываться извне сами по себе, чтобы ничего не поломалось. Условно говоря, если у нас есть автомобиль, его водитель, если только он не инженер-автомеханик, не должен иметь возможности вручную заставлять двигаться форсунки или чего-нибудь в этом роде, у водителя должен быть руль, газ, тормоз и коробка передач. Если неопытный водитель полезет копаться в моторе, он рискует все поломать. 

Также специальные средства работы с атрибутами классов позволяют, например, проверять их значения на корректность, прежде чем присваивать, выполнять логгирование (вести логи) и т.п. 

Скрывать атрибуты в питоне можно несколькими разными способами, часть которых мы уже рассмотрели. 

1. Использование псевдозакрытых атрибутов \_\_: к этим атрибутам можно обращаться извне, но сам факт наличия подчеркивания как бы намекает, что этого лучше не делать. 
2. Перегрузка методов \_\_getattr\_\_ & \_\_setattr\_\_.

Также можно использовать слоты. Что это такое? 

Обычно атрибуты у класса хранятся в специальном внутреннем словарике \_\_dict\_\_: такой словарик есть как у самого класса (статические атрибуты), так и у его экземпляров (динамические атрибуты). Словарь - он и в Африке словарь, мы можем в него в любой момент добавить новый атрибут, присвоив ему значение, а можем перезаписать имеющиеся (или удалить). Словарь в памяти хранится таким образом, чтобы можно было его расширять, у него нет фиксированного объема памяти. 

Атрибут \_\_slots\_\_ - это альтернатива словарю, если мы его объявляем, то словаря у нашего класса уже не будет. Он обычно объявляется на верхнем уровне определения класса, и в нем в виде списка или кортежа перечисляются атрибуты, которые нашему классу иметь дозволено:

In [ ]:
class Slots:
    __slots__ = ('a', 'b')
    def __init__(self, a, b):
        self.a = a
        self.b = b

Экземпляру такого класса нельзя будет создать новый атрибут, как бы мы ни хотели, а еще они гораздо быстрее работают, потому что у слотов заранее ограниченная (и известная) память. Мы с вами это тестили на занятии:

In [35]:
class Slots:
    __slots__ = ('a',)
    
class NoSlots:
    pass

def get_set_delete(obj):
    obj.a = 1
    obj.a
    del obj.a

In [36]:
s = Slots()
ns = NoSlots()

In [37]:
%timeit get_set_delete(s)

89.8 ns ± 0.0728 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [38]:
%timeit get_set_delete(ns)

101 ns ± 1.21 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Проблемы возникают при наследовании от таких классов. Если мы наследуемся от класса со слотами, то его дочерний класс все равно может получить словарь \_\_dict\_\_, и тогда весь смысл использования слотов пойдет насмарку. Следовательно, в дочернем классе тоже нужно обязательно прописывать слоты, тогда они сложатся со слотами родительского класса. 

А вот если мы хотим наследоваться сразу от двух классов со слотами, то не выйдет: 

In [2]:
class BaseOne:
    __slots__ = ('param',)
    
class BaseTwo:
    __slots__ = ('param2',)

class Child(BaseOne, BaseTwo):
    __slots__ = ()

TypeError: multiple bases have instance lay-out conflict

Обойти этот конфликт можно с помощью использования абстрактных классов. 

Основная мораль такова:

- Без переменной словаря dict экземплярам нельзя назначить атрибуты, не указанные в определении slots. Так можно ограничить создание левых атрибутов. 
- Атрибуты slots, объявленные в родительских классах, доступны в дочерних классах. Однако дочерние классы получат dict, если они не переопределяют slots. 
- Если класс определяет слот, также определенный в базовом классе, переменная экземпляра, определенная слотом базового класса, недоступна. Это приводит к неоднозначному поведению программы (не указывайте одинаковые атрибуты в классе-родителе и классе-ребенке). 
- Атрибуту slots может быть назначен любой нестроковый итерируемый объект. Могут использоваться словари, а значениям, соответствующим каждому ключу, может быть присвоено особое значение. 
- Множественное наследование от классов, которые оба имеют слоты, невозможно без абстрактных классов. 
- Слоты в подклассах бессмысленны, если их нет в суперклассе
- И наоборот. 
- Слоты реализуют атрибуты экземпляров, которые физически не хранятся в словарях пространств имен экземпляров и основаны на понятии дескрипторов атрибутов уровня класса. 
- Слоты работают быстрее словарей!

Далее рассмотрим переопределение нескольких магических методов:

    __setattr__ & __getattr__
    __setattribute & __getattribute__
    __set__, __get__, __delete__
  

#### Свойства (properties)

Мы с вами пользовались для делегирования переопределением методов \_\_setattr\_\_ & \_\_getattr\_\_. Эти методы вызываются только тогда, когда мы обращаемся к атрибуту экземпляра, которого у него нет. Например:

In [3]:
class A:
    def __init__(self, a):
        self.a = a
    def __getattr__(self, attr):
        print('getattr')

In [4]:
a = A(1)
a.a # обращаемся к атрибуту, который есть

1

In [5]:
a.b  # обращаемся к атрибуту, которого нет

getattr


Нужно быть осторожнее с этими методами, потому что если их неаккуратно переопределить, можно уйти в бесконечный цикл: если мы напишем в getattr getattr(self, attr), уйдем в рекурсию, можете проверить. 

Еще осторожнее нужно быть с методами \_\_getattribute\_\_ & \_\_setattribute\_\_, которые делают то же самое, но для всех атрибутов вообще, существуют они или нет. 

Если же нам нужно переопределить способ обращения к какому-то атрибуту класса (например, добавить проверку при присваивании), можно воспользоваться встроенной функцией property, которая обеспечивает интерфейс для атрибутов экземпляра класса. Сам метод принимает четыре аргумента:

    property(get, set, del, doc)
    
Как правило, если вы используете этот метод, вы точно захотите передать ему get, а дальше по убыванию: doc передается реже всего. Ни один из них по сути не обязательный, но get, наверное, хотелось бы передавать. 

Как это все реализуется:

In [71]:
class Person:
    def __init__(self):
        self.__name = ''
        
    def getname(self):
        print('getting name')
        return self.__name
        
    def setname(self, name):
        if name.isalpha():
            self.__name = name
    
    def delname(self):
        print('deleting name')
        del self.__name
    name = property(getname, setname, delname)  # property(get, set, del, doc)

In [72]:
p = Person()

In [73]:
p.name = 'Vasya'

In [74]:
p.name

getting name


'Vasya'

В каком порядке я определяю функции getname, setname & delname (а также как я их называю - можно называть их как угодно), неважно: важно только, в каком порядке я их передаю в метод property. 

Переменная name находится на верхнем уровне определения класса, на одном уровне с определениями методов. Но при этом сам атрибут name (точнее говоря, \_\_name) будет принадлежать конкретному экземпляру. Мы самому классу приписали свойство, которое теперь будет взаимодействовать со всеми его экземплярами, устанавливая им внутренний атрибут \_\_name. Это надо четко себе уяснить: name - не атрибут экземпляра. Это атрибут класса, который хранит в себе *дескриптор*. А дескриптор - это объект, который управляет внутренним атрибутом \_\_name. 

Да:  property - это разновидность дескриптора вообще. Давайте посмотрим про дескрипторы. 

#### Дескрипторы

Дескриптор - это такой объект питона, который имплементирует метод протокола дескриптора. Что такое протокол дескриптора? (descriptor protocol) Это то, каким образом питон работает с атрибутами объектов (классов или экземпляров). Мы с атрибутами что можем делать:

- Запрашивать их значение (\_\_get\_\_)
- Устанавливать их значение (\_\_set\_\_)
- Удалять их (\_\_delete\_\_)
- и устанавливать им имя и объект-хозяина (\_\_set\_name\_\_)

Все эти вещи и переопределяет дескриптор, чтобы можно было их делать каким-нибудь особенным образом. Дескрипторы бывают двух видов: data & non-data. Первый переопределяет метод \_\_set\_\_, а второй нет. 

Итак, как это все выглядит:

In [114]:
class Temperature:
    """descriptor"""
    def __get__(self, person, type=None):
        if hasattr(person, 'temp'):
            return person.temp
        raise AttributeError
    
    def __set__(self, person, value):
        if 30 <= value <= 42:
            person.temp = value
        else:
            print('Can\'t set!')

class Person:
    temperature = Temperature()

In [115]:
vasya = Person()

In [116]:
vasya.temperature = 43

Can't set!


In [117]:
vasya.temperature = 36

Дескриптор - это **отдельный класс**. Это и имеется в виду, когда мы говорим "объект". Когда мы определяем класс Person, у него заводим его статический атрибут, в который помещаем конкретный экземпляр класса "дескриптор". Теперь, когда мы будем писать vasya.temperature, мы на самом деле обращаемся к дескриптору, а не к атрибуту с целочисленным значением! А дескриптор уже вызывает нужные методы. То есть, что происходит, если совсем подробно:

    vasya = Person()  
    # у васи, как у экземпляра класса, образуется атрибут Temperature(), 
    то есть, создается конкретный объект-дескриптор. Никаких динамических атрибутов у васи нет!
    
    vasya.temperature 
    # мы обращаемся к дескриптору! У него в этот момент неявным образом вызывается метод __get__, который принимает что? 
    Сам экземпляр дескриптора (self), объект person = vasya, type, он же owner - это класс (мы можем его не передавать). 
    
    vasya.temperature = 36
    # мы опять обращаемся к дескриптору, но оператор присваивания заставляет вызваться метод __set__, который принимает опять
    сам экземпляр дескриптора, объект vasya и значение 36. Именно поэтому пишем person.temp = value: мы записываем значение уже в реальный динамический атрибут. 
    
    del vasya.temperature 
    # должен был бы вызвать метод __delete__, но мы его не определили. 

Дескрипторы, как и более простой их вариант property, обычно используются с синтаксисом декораторов (скоро посмотрим). Нужны они все для того же самого: для проверок, логов и всяческого разнообразного контроля над атрибутами, а еще чтоб жизнь малиной не казалась. :)

#### Статические методы и методы класса

Статические методы не принимают экземпляр класса: собственно говоря, эти методы могли бы существовать как отдельные функции, но часто их удобно запихивать в класс, чтобы а) не импортировать из модуля кучу имен б) лучшая структура программы в) переопределять их при наследовании!

Методы класса вместо экземпляра принимают сам класс. 

Допустим, у нас есть класс "Пицца", у которого хотим сделать метод, чтобы вычислять тупо площадь пиццы по ее диаметру: при этом начинка пиццы, которая нам важна для других методов, здесь не играет роли. Можно это сделать отдельной функцией:

In [10]:
import math

def pizza_area(d):
    return math.pi * (d / 2) ** 2

class Pizza:
    def __init__(self, *ingrs):
        self.ingredients = ingrs
    
    def price(self):
        return len(self.ingredients)  # это, конечно, глупость, но мне лень писать адекватно :)

В таком варианте, если мы захотим импортировать класс "Пицца" в другой скрипт, придется импортировать и функцию вместе с ним. Хочется включить метод в класс:

In [11]:
class Pizza:
    def __init__(self, *ingrs):
        self.ingredients = ingrs
    
    def price(self):
        return len(self.ingredients)  
    
    def pizza_area(d):
        return math.pi * (d / 2) ** 2

Такой код не будет работать:

In [8]:
pizza = Pizza('cheese', 'sausage', 'olives')
pizza.pizza_area(30)

TypeError: pizza_area() takes 1 positional argument but 2 were given

Будет работать такой:

In [12]:
Pizza.pizza_area(30)

706.8583470577034

Но, кстати говоря, во 2 питоне и такой не сработает (сейчас никто не пользуется 2 питоном, но мало ли...)

Тут-то и можно использовать статические методы:

In [13]:
class Pizza:
    def __init__(self, *ingrs):
        self.ingredients = ingrs
    
    def price(self):
        return len(self.ingredients)  
    
    def pizza_area(d):
        return math.pi * (d / 2) ** 2
    pizza_area = staticmethod(pizza_area)

In [14]:
pizza = Pizza('cheese', 'sausage', 'olives')
pizza.pizza_area(30)

706.8583470577034

Ура, все заработало и так. Теперь мы можем как хотим обращаться к нашему статическому методу, и возможно, с точки зрения логики будет правильно обращаться к нему все равно от экземпляра. 

Метод класса работает примерно так же, только еще принимает аргумент - ссылочку на сам класс. Можно устроить этакое:

In [15]:
class Pizza:
    def __init__(self, *ingrs):
        self.ingredients = ingrs
    
    def price(self):
        return len(self.ingredients)  
    
    def pizza_area(cls, d):
        return f'{cls.__name__}: {math.pi * (d / 2) ** 2}'
    pizza_area = classmethod(pizza_area)

In [16]:
pizza = Pizza('cheese', 'sausage', 'olives')
pizza.pizza_area(30)

'Pizza: 706.8583470577034'

Про super() и подробнее про дескрипторы еще поговорим на следующем занятии. 

Кто хочет разобраться получше, вот некоторые ссылки на материалы, которые я использовала:

- Ну во-первых, Лутц (т. II)
- [Про слоты](https://habr.com/ru/post/686220/)
- [Про дескрипторы](https://towardsdatascience.com/python-descriptors-and-how-to-use-them-5167d506af84)
- [Статические методы и методы класса + пицца](https://realpython.com/instance-class-and-static-methods-demystified/)

Если увидите синтаксис с @ - это как раз синтаксис с декораторами, на практике обычно используют именно его, но нам с вами нужно понимать, что за ним скрывается. 